In [ ]:
import pandas as pd
import json

In [ ]:
df = pd.read_csv("/content/dataset_tripadvisor-scraper-task_2024-11-28_13-04-38-742.csv")

<ipython-input-18-9888e4695cdd>:1: DtypeWarning: Columns (4,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,196,236,237,238,244,272,273,274,275,276,277,278,279,280,281,282,283,313,335,337,347,349,359,361,371,373,383,385,401,416,419,420,421,422,423,424,425,426,428,429,432,433,434,435,436,437,438,439,441,442,445,446,447,448,449,450,451,452,453,454,455,456,457,458,460,461,464,465,466,467,468,469,470,471,473,474,477,478,479,480,481,482,483,484,486,488,490,492,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,834,849) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/dataset_tripadvisor-scraper-task_2024-11-28_13-04-38-742.csv")


In [ ]:
# Check for null values
print("Null value summary:")
print(df.isnull().sum())

Null value summary:
address                     0
addressObj/city             7
addressObj/country          0
addressObj/postalcode     328
addressObj/state         1652
                         ... 
travelerChoiceAward      1413
type                        0
webUrl                      0
website                   243
whatsAppRedirectUrl      1625
Length: 855, dtype: int64


In [ ]:
# Fill missing values with defaults
df["address"] = df["address"].fillna("Unknown address")
df["aiReviewsSummary"] = df["aiReviewsSummary"].fillna("No summary available")
df["amenities/0"] = df["amenities/0"].fillna("No amenities mentioned")

# Replace missing data in all columns with a global default, if needed
df.fillna("No information available", inplace=True)

<ipython-input-20-f8a3eac72411>:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'No information available' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("No information available", inplace=True)


In [ ]:
df.head()

,address,addressObj/city,addressObj/country,addressObj/postalcode,addressObj/state,addressObj/street1,addressObj/street2,aiReviewsSummary,amenities/0,amenities/1,...,subtype/10,subtype/11,subtype/12,subtype/13,subtype/14,travelerChoiceAward,type,webUrl,website,whatsAppRedirectUrl
0,Ouarzazate Morocco,Ouarzazate,Morocco,No information available,No information available,No information available,No information available,No summary available,No amenities mentioned,No information available,...,No information available,No information available,No information available,No information available,No information available,No information available,ATTRACTION,https://www.tripadvisor.com/Attraction_Review-...,http://www.wonderful-marocco.weebly.com/,No information available
1,Marrakech 40000 Morocco,Marrakech,Morocco,40000,No information available,No information available,No information available,No summary available,No amenities mentioned,No information available,...,No information available,No information available,No information available,No information available,No information available,No information available,ATTRACTION,https://www.tripadvisor.com/Attraction_Review-...,http://www.morocco-adventure-holidays.com/day-...,No information available
2,Fez 30000 Morocco,Fez,Morocco,30000,No information available,No information available,No information available,No summary available,No amenities mentioned,No information available,...,No information available,No information available,No information available,No information available,No information available,No information available,ATTRACTION,https://www.tripadvisor.com/Attraction_Review-...,http://www.medinatours.net/,No information available
3,Agadir Morocco,Agadir,Morocco,No information available,No information available,No information available,No information available,No summary available,No amenities mentioned,No information available,...,No information available,No information available,No information available,No information available,No information available,No information available,ATTRACTION,https://www.tripadvisor.com/AttractionProductR...,No information available,No information available
4,Marrakech Morocco,Marrakech,Morocco,No information available,No information available,No information available,No information available,No summary available,No amenities mentioned,No information available,...,No information available,No information available,No information available,No information available,No information available,No information available,ATTRACTION,https://www.tripadvisor.com/AttractionProductR...,No information available,No information available


In [ ]:
dataset = []

In [ ]:
for _, row in df.iterrows():
        # Extract data for the question and answer
        city = row.get("addressObj/city", "unknown city")
        address = row.get("address", "unknown address")
        amenities = [row.get(f"amenities/{i}", "") for i in range(9) if pd.notna(row.get(f"amenities/{i}", ""))]
        top_amenity = amenities[0] if amenities else "no specific amenities mentioned"
        ai_summary = row.get("aiReviewsSummary", "No additional summary available.")

        # Formulate the question (input)
        input_question = f"I want to visit {city} in Morocco. Can you tell me more about it?"

        # Formulate the answer (output)
        output_answer = (
            f"Among the best places in {city} is this location: {address}. "
            f"It offers amenities such as {top_amenity}. {ai_summary}"
        )

        # Add to the QA dataset
        dataset.append({
            "input": input_question,
            "output": output_answer
        })


In [ ]:
dataset_trip_advisor = "tripadvisor_morocco_qa.json"
with open(dataset_trip_advisor, "w") as file:
    json.dump(dataset, file, indent=4)

print(f"QA dataset for Moroccan cities saved successfully as '{dataset_trip_advisor}'!")

QA dataset for Moroccan cities saved successfully as 'tripadvisor_morocco_qa.json'!


## AI part

In [ ]:
%pip install transformers

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text2text-generation", model="atlasia/Terjman-Ultra")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/850 [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
# Translation function
def translate_to_darija_pipeline(text):
    # Use the pipeline to generate translation
    result = pipe(text, max_length=512, truncation=True)

    # Extract the translated text from the result
    print(result)
    translated_text = result[0]["generated_text"]
    return translated_text


In [ ]:
test= "my name is salma and i live in Morocco"


In [ ]:
result = translate_to_darija_pipeline(test)

[{'generated_text': 'سميتي سلما وأعيش فالمغرب'}]


In [ ]:
result

'سميتي سلما وأعيش فالمغرب'

In [ ]:
%pip install tqdm

In [ ]:
from tqdm import tqdm
import json

In [ ]:
with open("/content/tripadvisor_morocco_qa.json", "r", encoding="utf-8") as file:
    data = json.load(file)

In [ ]:
# Translate the dataset
for item in tqdm(data, desc="tripadvisor_morocco_qa"):
    item["input"] = translate_to_darija_pipeline(item["input"])
    item["output"] = translate_to_darija_pipeline(item["output"])

tripadvisor_morocco_qa:   0%|          | 0/3312 [00:00<?, ?it/s]

[{'generated_text': 'بغيت نزور أوارزازات في المغرب. واش تقدر تقولي لي مزيد على هادشي؟'}]


tripadvisor_morocco_qa:   0%|          | 1/3312 [00:06<5:57:22,  6.48s/it]

[{'generated_text': 'من بين أحسن الأماكن في أورزازات هو هذا الموقع: أورزازات المغرب. هو كيوفر مرافق كمان لا توجد مرافق. لا توجد ملخصة'}]
[{'generated_text': 'بغيت نزور مراكش في المغرب. واش تقدر تقولي لي مزيد على هادشي؟'}]


tripadvisor_morocco_qa:   0%|          | 2/3312 [00:12<5:57:07,  6.47s/it]

...

[{'generated_text': 'من بين أحسن الأماكن في مراكش هو هذا الموقع: ارسط أوزال باب دوكالا، مراكش 40000 المغرب. كيوفر مرافق كمناسبات مثل الإنترنت. ماعندها ملخص متاح'}]
[{'generated_text': 'بغيت نزور مراكش في المغرب. واش تقدر تقولي لي مزيد على هادشي؟'}]


tripadvisor_morocco_qa:  31%|███▏      | 1041/3312 [1:59:54<4:29:06,  7.11s/it]

[{'generated_text': 'من بين احسن اماكن في مراكش هو هاد الموقع: App 10 imm 7 hay achrifiya, مراكش 40000 المغرب. كيعرض مرافق كايسمى ماشي مرافق متذكرة. ماشي ملخص متاح'}]
[{'generated_text': 'بغيت نزور مراكش في المغرب. واش تقدر تقولي لي مزيد على هادشي؟'}]


tripadvisor_morocco_qa:  31%|███▏      | 1042/3312 [2:00:01<4:24:58,  7.00s/it]

[{'generated_text': 'من بين أحسن الأماكن في مراكش هو هذا الموقع: شارع باب دوكالا، مراكش 40000 المغرب. هو كيوفر مرافق كمان لا مرافق مذكورة. لا ملخص متاح'}]


In [ ]:
# Save the current state of the dataset
with open("english_to_darija_trip_Advisor.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print("Progress saved.")

Progress saved.


## preprocessing data

In [ ]:
import re

def preprocess_morocco_qa_txtfile(file_path):
    """
    Preprocess TripAdvisor Morocco QA dataset with robust question-answer identification.

    Args:
        file_path (str): Path to the text file containing raw data

    Returns:
        list: List of tuples containing (verified question, verified answer)
    """
    processed_data = []

    with open(file_path, 'r', encoding='utf-8') as file:
        # Read entire file content
        content = file.read()

        # Use regex to find all instances of generated_text
        generated_texts = re.findall(r"\[{'generated_text': '(.*?)'}]", content)

        # Process texts by identifying correct question-answer pairs
        for i in range(len(generated_texts)):
            # Check if current text is a question
            if generated_texts[i].startswith('بغيت'):
                # Look for the next text that starts with "من بين أحسن"
                for j in range(i+1, len(generated_texts)):
                    if generated_texts[j].startswith('من بين أحسن'):
                        processed_data.append((generated_texts[i], generated_texts[j]))
                        break

    return processed_data


# Bonus: Add validation function
def validate_dataset(dataset):
    """
    Validate the preprocessed dataset.

    Args:
        dataset (list): List of (question, answer) tuples

    Returns:
        bool: True if all entries are valid, False otherwise
    """
    for q, a in dataset:
        if not q.startswith('بغيت') or not a.startswith('من بين أحسن'):
            return False
    return True




In [ ]:
# Example usage
dataset = preprocess_morocco_qa_txtfile('/content/dataset_darija_eng.txt')
if validate_dataset(dataset):
  print(f"Successfully processed {len(dataset)} question-answer pairs")
else:
  print("Dataset processing failed validation")

Successfully processed 733 question-answer pairs


In [ ]:
dataset

[('بغيت نزور أوارزازات في المغرب. واش تقدر تقولي لي مزيد على هادشي؟',
  'من بين أحسن الأماكن في أورزازات هو هذا الموقع: أورزازات المغرب. هو كيوفر مرافق كمان لا توجد مرافق. لا توجد ملخصة'),
 ('بغيت نزور مراكش في المغرب. واش تقدر تقولي لي مزيد على هادشي؟',
  'من بين أحسن الأماكن في مراكش هو هذا الموقع: مراكش 40000 المغرب. هو كيوفر مرافق كمان لا مرافق متذكرة. لا ملخص متاح'),
 ('بغيت نزور فيز في المغرب. واش تقدر تقولي لي مزيد على هادشي؟',
  'من بين أحسن الأماكن ففيص فهاد الموقع: فيص 30000 المغرب. كيعرض مرافق كلي ماشي مرافق متذكورة. ماشي ملخص متوفر'),
 ('بغيت نزور أغادير فالمغرب. واش تقدر تقولي لي مزيد على هادشي؟',
  'من بين أحسن الأماكن في أغدير هو هاد الموقع: أغدير المغرب. كاين مرافق كمان ماشي ماشي. ماشي ملخص متاح'),
 ('بغيت نزور مراكش في المغرب. واش تقدر تقولي لي مزيد على هادشي؟',
  'من بين أحسن الأماكن في مراكش هو هذا الموقع: مراكش المغرب. هو كيوفر مرافق كمان لا مرافق متذكرة. لا ملخص متاح'),
 ('بغيت نزور مراكش في المغرب. واش تقدر تقولي لي مزيد على هادشي؟',
  'من بين أحسن الأماكن في مراك

In [ ]:
# Save the current state of the dataset
with open("cleaned_dataset.txt", "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

print("Progress saved.")

Progress saved.


In [ ]:
def save_as_plain_text(dataset, file_path):
    """
    Saves a dataset of tuples (question, answer) as a plain text file.

    Args:
    - dataset: List of tuples (question, answer).
    - file_path: Path to save the plain text file.
    """
    with open(file_path, 'w', encoding='utf-8') as file:
        for question, answer in dataset:
            file.write(f"{question.strip()}\n")
            file.write(f"{answer.strip()}\n\n")  # Add a blank line between entries


In [ ]:
# Save as plain text
save_as_plain_text(dataset, '/content/dataset.txt')

## Data Quality

In [ ]:
dataset = [
    {
        "input":"What are the must-see attractions in Marrakech for history lovers?",
        "output":"History lovers in Marrakech should visit the Saadian Tombs for a glimpse of royal history, Bahia Palace for its intricate architecture, and Koutoubia Mosque, the largest mosque in Marrakech."
    },
    {
        "input":"I’m planning to visit Casablanca in May. What unique activities should I try?",
        "output":"In May, enjoy a walk along the Ain Diab Corniche, explore the Morocco Mall for its unique aquarium and luxury shopping, and dine at Rick's Café, inspired by the movie Casablanca."
    },
    {
        "input":"What are the best places to visit in Fes for a cultural experience?",
        "output":"In Fes, explore the Al-Attarine Madrasa for its stunning tilework, visit the Tanneries to learn about leather-making, and wander through the vibrant souks in the Fes Medina, a UNESCO World Heritage site."
    },
    {
        "input":"What are the top activities to do in Essaouira for adventure seekers?",
        "output":"Adventure seekers in Essaouira can try kite surfing along the windy beaches, take a horseback ride along the coast, and explore the surrounding dunes on a quad bike."
    },
    {
        "input":"What are some unique things to do in Rabat with kids?",
        "output":"With kids in Rabat, visit Rabat Zoo to see exotic animals, enjoy a picnic at the Bouregreg Marina, and explore the Children’s Science Museum for interactive exhibits."
    },
    {
        "input":"What are the top three activities to do in Tangier in September?",
        "output":"In September, visit the Cape Spartel Lighthouse for stunning coastal views, explore the Caves of Hercules, and take a walk along the Tangier Kasbah to enjoy the vibrant local culture."
    },
    {
        "input":"What are the best off-the-beaten-path attractions in Chefchaouen?",
        "output":"In Chefchaouen, hike to the Akchour Waterfalls for a serene escape, visit the Ras El Maa viewpoint, and explore the blue streets away from the main square for a quieter experience."
    },
    {
        "input":"I want to visit Agadir. What are the best wellness activities to try?",
        "output":"In Agadir, relax with a traditional hammam experience, take yoga classes at a beachside retreat, and visit Crocoparc for a peaceful day surrounded by lush gardens and crocodiles."
    },
    {
        "input":"What are the top food experiences to try in Marrakech?",
        "output":"In Marrakech, dine at a rooftop restaurant in the Medina for tajines, visit Dar Yacout for an upscale Moroccan meal, and try street food like msemen and harira soup in Jemaa el-Fnaa square."
    },
    {
        "input":"What are the must-visit spots in Meknes for a one-day trip?",
        "output":"For a one-day trip in Meknes, visit Bab Mansour gate, the Mausoleum of Moulay Ismail, and the Volubilis Roman ruins located just outside the city."
    },
    {
        "input":"What are the best stargazing spots in Merzouga?",
        "output":"Merzouga offers incredible stargazing opportunities. Book a night in a Berber desert camp, climb to the top of Erg Chebbi dunes for a clear view, or enjoy stargazing during a camel trek."
    },
    {
        "input":"What are the most scenic viewpoints in Dakhla?",
        "output":"In Dakhla, visit Dragon Island for panoramic views, explore the White Dune for a breathtaking desert-meets-ocean scene, and enjoy the lagoon views while kite surfing."
    },
    {
        "input":"What should I prioritize if visiting Ouarzazate for one day?",
        "output":"In one day, visit Ait Ben Haddou kasbah for stunning historical views, tour the Atlas Film Studios, and check out the Taourirt Kasbah in the city center."
    },
    {
        "input":"What are the top hiking spots around Ifrane?",
        "output":"Near Ifrane, hike the Cedar Forest to spot Barbary monkeys, explore the Michlifen ski area trails, and visit Dayet Aoua Lake for scenic views and peaceful trails."
    },
    {
        "input":"What are the must-visit places in Tétouan for art lovers?",
        "output":"Art lovers should visit the National Institute of Fine Arts, explore the medina’s artisan workshops, and check out the Archaeological Museum for exhibits on local history and culture."
    },
    {
        "input":"What are the best things to do in Oujda during spring?",
        "output":"In spring, enjoy the blooming flowers at Lalla Aicha Park, visit the Grand Mosque for a spiritual experience, and explore Sidi Yahya Oasis for a peaceful retreat."
    },
    {
        "input":"What are the top experiences in Asilah for photographers?",
        "output":"Photographers will love capturing Asilah’s colorful medina murals, the dramatic views from the ramparts at sunset, and the pristine beauty of Paradise Beach."
    },
    {
        "input":"What are the best ways to explore the desert near Zagora?",
        "output":"In Zagora, take a camel trek into the dunes, enjoy a 4x4 desert tour, and spend the night at a luxury camp to experience traditional Moroccan hospitality under the stars."
    },
    {
        "input":"What are the best culinary experiences in Fes?",
        "output":"In Fes, take a cooking class to learn how to make tajine, try pastilla at a local restaurant, and visit the spice markets in the medina for an authentic food experience."
    },
    {
        "input":"What are the must-see coastal towns in Morocco?",
        "output":"Must-see coastal towns include Essaouira for its historic charm, Agadir for its beaches and resorts, and El Jadida for its Portuguese cisterns and relaxed vibe."
    },
    {
        "input":"What are the top winter activities in Ifrane?",
        "output":"In winter, enjoy skiing or snowboarding at Michlifen, take a scenic walk in the snowy Cedar Forest, and warm up with traditional Moroccan tea at a local café."
    },
    {
        "input":"What are the must-visit museums in Rabat?",
        "output":"In Rabat, visit the Mohammed VI Museum of Modern and Contemporary Art, the Archaeological Museum, and the Oudaya Museum for a blend of history and art."
    },
    {
        "input":"What are the best markets to visit in Marrakech?",
        "output":"The best markets include the bustling Souk Semmarine for spices and textiles, the Rahba Kedima market for exotic goods, and the Mellah for jewelry and antiques."
    },
    {
        "input":"What are the best sunset spots in Tangier?",
        "output":"Catch stunning sunsets from Cape Spartel, the rooftop cafés in the Kasbah, or the beach promenade along Tangier Bay."
    },
    {
        "input":"What are the top attractions in Azrou for nature enthusiasts?",
        "output":"In Azrou, visit the famous Cedar Forest to spot monkeys, hike around the Ain Leuh waterfalls, and explore the volcanic rock formations nearby."
    },
    {
        "input":"What are the most iconic landmarks in Casablanca?",
        "output":"Iconic landmarks in Casablanca include the Hassan II Mosque, the Art Deco buildings in downtown, and the Royal Palace, which can be viewed from outside."
    },
    {
        "input":"What are the top day trips from Marrakech?",
        "output":"Popular day trips from Marrakech include visiting the Ouzoud Waterfalls, exploring the Ourika Valley, and touring Ait Ben Haddou, a UNESCO World Heritage site."
    },
    {
        "input":"What are the best ways to experience Moroccan music and culture in Fes?",
        "output":"In Fes, attend a live performance during the Fes Festival of World Sacred Music, visit the Dar Batha Museum, and listen to Andalusian music at a traditional riad."
    },
    {
        "input":"What are the most family-friendly activities in Agadir?",
        "output":"Family-friendly activities in Agadir include visiting the Agadir Crocoparc, enjoying water slides at Atlantica Parc, and relaxing on Agadir Beach with its calm waters."
    },
    {
        "input":"What are the must-visit natural attractions near Dakhla?",
        "output":"Near Dakhla, explore the White Dune, visit the freshwater Asnaa hot springs, and take a boat ride to see flamingos in the lagoon."
    },
    {
        "input":"What are the best things to do in Chefchaouen during spring?",
        "output":"In spring, explore the blooming gardens around the Ras El Maa waterfall, hike the trails around Akchour, and enjoy the mild weather while strolling through the medina."
    },
    {
        "input":"What are the best cultural experiences in Meknes?",
        "output":"In Meknes, explore the Dar Jamai Museum, visit the vibrant medina, and learn about the city’s history at the Mausoleum of Moulay Ismail."
    },
    {
        "input":"What are the best beaches in Tangier?",
        "output":"Top beaches in Tangier include Malabata Beach for relaxation, Achakar Beach for its dramatic cliffs, and the beaches near Cape Spartel for unspoiled nature."
    },
    {
        "input":"What are the most unique architectural landmarks in Marrakech?",
        "output":"Unique landmarks in Marrakech include the Koutoubia Mosque, the Ben Youssef Madrasa for intricate designs, and the El Badi Palace ruins."
    }
]


In [ ]:
len(dataset)

34

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text2text-generation", model="atlasia/Terjman-Ultra")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/850 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.74G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.9k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/4.23k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
# Translation function
def translate_to_darija_pipeline(text):
    # Use the pipeline to generate translation
    result = pipe(text, max_length=512, truncation=True)

    # Extract the translated text from the result
    print(result)
    translated_text = result[0]["generated_text"]
    return translated_text

In [ ]:
from tqdm import tqdm
import json

# Translate the dataset
for item in tqdm(dataset, desc="tripadvisor_morocco_qa"):
    item["input"] = translate_to_darija_pipeline(item["input"])
    item["output"] = translate_to_darija_pipeline(item["output"])

tripadvisor_morocco_qa:   0%|          | 0/34 [00:00<?, ?it/s]

[{'generated_text': 'شنو هي الجوائز اللي خاص تلاقيها في مراكش من اجل عشاق التاريخ؟'}]


tripadvisor_morocco_qa:   3%|▎         | 1/34 [00:10<05:45, 10.48s/it]

[{'generated_text': 'لي حببو التاريخ فالمراكش خاصهم بزيارة القبور السعدية باش يتعرفوا على التاريخ الملكي، قصر باهيا بسبب الهندسة المعمارية المعقدة، ومسجد كوتوبيا، أكبر مسجد فالمراكش.'}]
[{'generated_text': 'أنا كايخطط باش نزور الدار البيضاء فمايو. شنو هي الأنشطة الفريدة لي فيتوجب عليا نحاولها؟'}]


tripadvisor_morocco_qa:   6%|▌         | 2/34 [00:22<06:05, 11.41s/it]

...

[{'generated_text': 'فمكنس، إستكشاف متحف دارجامي، زيارة المدينة الحيوية، وتعلم عن تاريخ المدينة في ضريح مولاي إسماعيل.'}]
[{'generated_text': 'شنو هي أحسن شواطئ طنجة؟'}]


tripadvisor_morocco_qa:  97%|█████████▋| 33/34 [05:01<00:08,  8.40s/it]

[{'generated_text': 'الشواطئ الرائدة في طنجة هي شاطئ مالاباتا للتسريح، شاطئ أشكار للفلاجات الدراماتيكية، والشواطئ القريبة من كيب سبارتل للطبيعة غير الملوثة.'}]
[{'generated_text': 'شنو هي الأماكن المعماريّة الأكثر فريدة في مراكش؟'}]


tripadvisor_morocco_qa: 100%|██████████| 34/34 [05:09<00:00,  9.09s/it]

[{'generated_text': 'معالم فريدة في مراكش هي المسجد الكوطوبي، مدرسة بن يوسف للتصاميم المعقدة، وأثار قصر البادي.'}]


In [ ]:
dataset

[{'input': 'شنو هي الجوائز اللي خاص تلاقيها في مراكش من اجل عشاق التاريخ؟',
  'output': 'لي حببو التاريخ فالمراكش خاصهم بزيارة القبور السعدية باش يتعرفوا على التاريخ الملكي، قصر باهيا بسبب الهندسة المعمارية المعقدة، ومسجد كوتوبيا، أكبر مسجد فالمراكش.'},
 {'input': 'أنا كايخطط باش نزور الدار البيضاء فمايو. شنو هي الأنشطة الفريدة لي فيتوجب عليا نحاولها؟',
  'output': 'فمايو، تمتع بمشي على طول قرية عين دياب، استكشف مركز التسوق المغرب لأكواريم فريدة وأسواق فاخرة، وتناول العشاء في مقهى ريك، مستوحى من الفيلم كازابلانكا.'},
 {'input': 'شنو هي أحسن الأماكن للزيارة فيس لتجربة ثقافية؟',
  'output': 'فيس، قم بزيارة مدرسة الأعتارين لأعمالها البلاطية المذهلة، وزر المصانع لتعلم عن صنع الجلد، وتتجول في الصوكات النابضة بالحياة في مدينة فيس، موقع التراث العالمي لليونسكو.'},
 {'input': 'شنو هي الأنشطة الأهم اللي يمكن القيام بها في إساويرا بالنسبة لمتساعدي المغامرات؟',
  'output': 'اللي كايبغوا المغامرات في إساويرا يمكن ليهم يجراوو كايت سيرف على الشواطئ اللي فيها الرياح، ويقومو بركوب الخيل على طول الساحل